In [1]:
import imp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import axelrod as axl
import axelrod.interaction_utils as iu

import testzd as zd

C, D = axl.Action.C, axl.Action.D

In [2]:
parameters = imp.load_source('parameters', 'data/raw/parameters.py')

# Extortionate zero determinant.

In [1], given a match between 2 memory one strategies the concept of Zero Determinant strategies is introduced. It was showed that a player $p\in\mathbb{R}^4$ against a player $q\in\mathbb{R}^4$ could force a linear relationship between the scores.

Assuming the following:

- The utilities for player $p$: $S_x = (R, S, T, P)$ and for player $q$: $S_y = (R, T, S, P)$.
- The normalised long run score for player $p$: $s_x$ and for player $q$: $s_y$.
- Given $p=(p_1, p_2, p_3, p_4)$ a transformed (but equivalent) vector: $\tilde p=(p_1 - 1, p_2 - 1, p_3, p_4)$, similarly: $\tilde q=(1 - q_1, 1 - q_2, q_3, q_4)$

The main result of [1] is that:

if $\tilde p = \alpha S_x + \beta S_y + \gamma 1$ **or** if $\tilde q = \alpha S_x + \beta S_y + \gamma 1$ then:

$$
\alpha s_x + \beta s_y + \gamma 1 = 0
$$

where $\alpha, \beta, \gamma \in \mathbb{R}$

As an example consider the `extort-2` strategy defined in [2]. This is given by:

$$p=(8/9, 1/2, 1/3, 0)$$

Let us use the `Axelrod` library [4, 5] to simulate some matches, here it is against some of the best strategies in the Axelrod library:

In [3]:
extort2 = axl.ZDExtort2()
players = (extort2, axl.EvolvedFSM16())
axl.seed(0)
match = axl.Match(players, turns=parameters.TURNS)
interactions = match.play()
scores = match.final_score_per_turn()
scores[0] / scores[1]

1.4240943070730305

In [4]:
players = (extort2, axl.EvolvedANN5())
axl.seed(0)
match = axl.Match(players, turns=parameters.TURNS)
interactions = match.play()
scores = match.final_score_per_turn()
scores[0] / scores[1]

1.0123456790123457

In [5]:
players = (extort2, axl.PSOGamblerMem1())
axl.seed(0)
match = axl.Match(players, turns=parameters.TURNS)
interactions = match.play()
scores = match.final_score_per_turn()
scores[0] / scores[1]

1.3522765038218678

We see that `extort2` beats all these strategies. 

In [1], in fact a specific type of Zero determinant strategy is considered, indeed if: $p_4=0$ then the relationship $\chi = S_X / S_Y$ holds where $\chi = \frac{-\beta}{\alpha}$ so that the $S_X$ will be at $\chi$ times bigger than $S_Y$ as long as $\chi > 1$.

In the case of `extort2` can we reverse engineer $\alpha$ and $\beta$?

This is equivalent to finding $\alpha, \beta, \gamma \in \mathbb{R}$ such that $\tilde p = \alpha S_x + \beta S_y + \gamma 1$.

Recall that $\tilde p, S_x, S_y, 1\in\mathbb{R}^{4\times 1}$ so this corresponds to a linear system of 4 equations on three variables.

$$\tilde p=Cx$$

Where:

$$
C = \begin{pmatrix}S_x, S_y, 1\end{pmatrix}\in\mathbb{R}^{4\times 3}
$$

As an example consider the `extort-2` strategy defined in [2]. This is given by:

$$p=(8/9, 1/2, 1/3, 0)$$

it is defined to ensure:

$$
\begin{aligned}
\alpha s_x - P &= 2(s_y - P)\\
\alpha s_x - 2s_y + P&=0\\
\end{aligned}
$$

Let us solve $Cx=\tilde p$, we will do this using a least squares approach (which is numerically more efficient than inverting $C$ [3]):

In [6]:
p = np.array([8 / 9, 1 / 2, 1 / 3, 0])
x, SSError = zd.compute_least_squares(p)
alpha, beta, gamma = x
chi = -beta / alpha
chi

2.000000000000003

We see that in the case of `extort2` we get a value of $\chi > 2$ however none of the score ratios above was 2. This is because, in fact the high performing strategies against which `extort2` was playing are not memory one. They still lost their match though.

Because of the dimension of $C$: $4\times 3$ the linear system might not necessarily have a solution, this would be the case if the strategy is not Zero determinant. Thus, using a least squares minimisation approach we are in fact minimising the following value:

$$\text{SSError}=\|Cx-\bar p\|_2^2$$

This value: $\text{SSError}$, in fact gives us a measure of how far from being a Zero determinant strategy a given strategy vector $p$ is.

While all strategies are not necessarily memory one: so do not necessarily have a representation as a 4 dimensional vector. There transition rates from all states to any action can still be measured.

Let us see how this works, using the 3 strategies above:

In [7]:
def get_p_from_interactions(interactions):
    vectors = []
    for state_counter in iu.compute_state_to_action_distribution(interactions):
        p = []
        for state in ((C, C), (C, D), (D, C), (D, D)):
            try:
                p.append(state_counter[(state, C)] / (state_counter[(state, C)]  + state_counter[(state, D)] ) )
            except ZeroDivisionError:
                p.append(np.NaN)
        vectors.append(p)
    return vectors

In [8]:
players = (extort2, axl.EvolvedFSM16())
axl.seed(0)
match = axl.Match(players, turns=parameters.TURNS)
interactions = match.play()
p = get_p_from_interactions(interactions=interactions)[1]

In [9]:
p

[0.3746031746031746,
 0.47244094488188976,
 0.5443786982248521,
 0.5167464114832536]

We can check how close this strategy is to being ZD:

In [10]:
x, SSError = zd.compute_least_squares(p)
SSError

0.10781562347847042

In [11]:
players = (extort2, axl.EvolvedANN5())
axl.seed(0)
match = axl.Match(players, turns=parameters.TURNS)
interactions = match.play()
p = get_p_from_interactions(interactions=interactions)[1]
x, SSError = zd.compute_least_squares(p)
SSError

nan

This strategy in fact does not visit all states so it is not possible to give a valid calculation:

In [12]:
p

[1.0, nan, 0.8, 0.0]

In [13]:
players = (extort2, axl.PSOGambler2_2_2())
axl.seed(0)
match = axl.Match(players, turns=parameters.TURNS)
interactions = match.play()
p = get_p_from_interactions(interactions=interactions)[1]
x, SSError = zd.compute_least_squares(p)
SSError

0.08277060885461741

So it seems that the `PSOGambler2_2_2` is "more" ZD than the other two. Note: it is certainly not an extortionate strategy as $p_4 > 0$:

In [14]:
p

[0.10476190476190476,
 0.5183098591549296,
 0.0017331022530329288,
 0.5051975051975052]

We can actually classify all potential extortionate strategies which is Figure 1 of the paper.

The paper extends this work to consider a LARGE number of strategies, and identifies if and when strategies actually exhibit extortionate behaviour.

We note that the strategies that exhibit strong evolutionary fitness are ones that are able to adapt their behaviour: they do not extort strong strategies (thus cooperation evolves) but they do extort weaker ones. For example, here is a list of strategies against which `EvolvedFSM16` is close to being ZD ($\text{SSError} < 0.05$) and is close to being extortionate: ($p_4 < 0.05$):

In [15]:
for opponent in parameters.PLAYER_GROUPS["full"]:
    players = (axl.EvolvedFSM16(), opponent)
    axl.seed(0)
    match = axl.Match(players, turns=parameters.TURNS)
    interactions = match.play()
    p = get_p_from_interactions(interactions=interactions)[0]
    x, SSError = zd.compute_least_squares(p)
    if SSError < 0.05 and p[3] < 0.05:
        alpha, beta, gamma = x
        scores = match.final_score_per_turn()
        print(f"vs {opponent}, chi={round(-beta / gamma, 2)}, S_X/S_Y={round(scores[0] / scores[1], 2)}")

vs AntiCycler, chi=0.89, S_X/S_Y=133.47
vs Arrogant QLearner, chi=1.27, S_X/S_Y=1.37
vs Bush Mosteller: 0.5, 0.5, 3.0, 0.5, chi=0.59, S_X/S_Y=2.47
vs Cautious QLearner, chi=1.27, S_X/S_Y=1.37
vs Colbert, chi=1.24, S_X/S_Y=5.78
vs Hesitant QLearner, chi=1.27, S_X/S_Y=1.37
vs Knowledgeable Worse and Worse, chi=1.25, S_X/S_Y=4.12
vs Prober 4, chi=0.79, S_X/S_Y=1.0
vs Random: 0.5, chi=0.64, S_X/S_Y=5.54
vs Risky QLearner, chi=1.27, S_X/S_Y=1.37
vs Stochastic Cooperator, chi=0.71, S_X/S_Y=3.14
vs ThueMorseInverse, chi=0.95, S_X/S_Y=5.94
vs Tranquilizer, chi=2.37, S_X/S_Y=1.01
vs Tullock: 11, chi=1.18, S_X/S_Y=0.96
vs Worse and Worse, chi=0.97, S_X/S_Y=2.4
vs Worse and Worse 2, chi=0.74, S_X/S_Y=0.93
vs ZD-Mem2, chi=0.64, S_X/S_Y=1.5


This work shows here that not only is there a mathematical basis for suspicion: the calculation of $\text{SSError}$ but that some high performing strategies seem to exhibit suspicious behaviour that allows them to adapt.

## References

[1] Press, William H., and Freeman J. Dyson. "Iterated Prisoner’s Dilemma contains strategies that dominate any evolutionary opponent." Proceedings of the National Academy of Sciences 109.26 (2012): 10409-10413

[2] Stewart, Alexander J., and Joshua B. Plotkin. "Extortion and cooperation in the Prisoner’s Dilemma." Proceedings of the National Academy of Sciences 109.26 (2012): 10134-10135.

[3] Golub, Gene H., and Charles F. Van Loan. Matrix computations. Vol. 3. JHU Press, 2012.

[4] The Axelrod project developers. Axelrod: v4.2.0. 2016. http://doi.org/10.5281/zenodo.1252994

[5] Knight, Vincent, et al. "An Open Framework for the Reproducible Study of the Iterated Prisoner’s Dilemma." Journal of Open Research Software 4.1 (2016).